In [1]:
import json 
import numpy as np 
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Embedding, GlobalAveragePooling1D
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.preprocessing import LabelEncoder

**data preprocessing**

In [2]:
with open(r"D:\my_projects\62A\Intent.json") as file:
    data = json.load(file)
    
training_sentences = []
training_labels = []
labels = []
responses = []


for intent in data['intents']:
    for pattern in intent['text']:
        training_sentences.append(pattern)
        training_labels.append(intent['intent'])
    responses.append(intent['responses'])
    
    if intent['intent'] not in labels:
        labels.append(intent['intent'])
        
num_classes = len(labels)

In [3]:

lbl_encoder = LabelEncoder()
lbl_encoder.fit(training_labels)
training_labels = lbl_encoder.transform(training_labels)

In [4]:
vocab_size = 1000
embedding_dim = 16
max_len = 20
oov_token = "<OOV>"

tokenizer = Tokenizer(num_words=vocab_size, oov_token=oov_token)
tokenizer.fit_on_texts(training_sentences)
word_index = tokenizer.word_index
sequences = tokenizer.texts_to_sequences(training_sentences)
padded_sequences = pad_sequences(sequences, truncating='post', maxlen=max_len)

**building the model neural network architecture**

In [5]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim, input_length=max_len))
model.add(GlobalAveragePooling1D())
model.add(Dense(16, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy', 
              optimizer='adam', metrics=['accuracy'])

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 16)            16000     
                                                                 
 global_average_pooling1d (G  (None, 16)               0         
 lobalAveragePooling1D)                                          
                                                                 
 dense (Dense)               (None, 16)                272       
                                                                 
 dense_1 (Dense)             (None, 16)                272       
                                                                 
 dense_2 (Dense)             (None, 22)                374       
                                                                 
Total params: 16,918
Trainable params: 16,918
Non-trainable params: 0
____________________________________________________

**fitting the model**

In [7]:
epochs = 50
history = model.fit(padded_sequences, np.array(training_labels), epochs=epochs)

Epoch 1/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0210 - accuracy: 1.0000
Epoch 2/50
5/5 [==============================] - 0s 12ms/step - loss: 0.0208 - accuracy: 1.0000
Epoch 3/50
5/5 [==============================] - 0s 5ms/step - loss: 0.0207 - accuracy: 1.0000
Epoch 4/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0205 - accuracy: 1.0000
Epoch 5/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0203 - accuracy: 1.0000
Epoch 6/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0202 - accuracy: 1.0000
Epoch 7/50
5/5 [==============================] - 0s 8ms/step - loss: 0.0201 - accuracy: 1.0000
Epoch 8/50
5/5 [==============================] - 0s 14ms/step - loss: 0.0200 - accuracy: 1.0000
Epoch 9/50
5/5 [==============================] - 0s 15ms/step - loss: 0.0199 - accuracy: 1.0000
Epoch 10/50
5/5 [==============================] - 0s 6ms/step - loss: 0.0197 - accuracy: 1.0000
Epoch 11/50
5/5 [===================

**saving the model**

In [8]:
model.save("chat_model")

import pickle

# to save the fitted tokenizer
with open('tokenizer.pickle', 'wb') as handle:
    pickle.dump(tokenizer, handle, protocol=pickle.HIGHEST_PROTOCOL)
    
# to save the fitted label encoder
with open('label_encoder.pickle', 'wb') as ecn_file:
    pickle.dump(lbl_encoder, ecn_file, protocol=pickle.HIGHEST_PROTOCOL)

INFO:tensorflow:Assets written to: chat_model\assets


INFO:tensorflow:Assets written to: chat_model\assets
